# Creating PDF Reports with Pandas, Jinja and WeasyPrint
[http://pbpython.com/pdf-reports.html](http://pbpython.com/pdf-reports.html) 

First of all I've mocked some data on Mockaroo

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/sales_report.csv", sep=',', encoding='utf-8')
df.head()

,Account,Name,Company,Manager,Product,Quantity,Price
0,383834,Izaak Tattersall,"Bahringer, Balistreri and Dickens",Gloria Osbaldeston,Pork - Ham Hocks - Smoked,1,$453.55
1,656536,Enoch Nasey,Feeney-Aufderhar,Johnny Pearde,Wine - Chenin Blanc K.w.v.,2,$760.73
2,627574,Clifford Dursley,Moen Inc,Earle Geaney,Raisin - Golden,3,$320.12
3,707250,Peta Juliff,Pacocha and Sons,Paloma Sheavills,Carbonated Water - Raspberry,4,$132.22
4,532894,Shaina Estick,Block LLC,Giovanni Schwier,Steampan Lid,5,$520.72


In [3]:
price_to_num = lambda x: float(x.replace("$",""))
df['Price'] = df['Price'].map(price_to_num)

In [4]:
df.head()

,Account,Name,Company,Manager,Product,Quantity,Price
0,383834,Izaak Tattersall,"Bahringer, Balistreri and Dickens",Gloria Osbaldeston,Pork - Ham Hocks - Smoked,1,453.55
1,656536,Enoch Nasey,Feeney-Aufderhar,Johnny Pearde,Wine - Chenin Blanc K.w.v.,2,760.73
2,627574,Clifford Dursley,Moen Inc,Earle Geaney,Raisin - Golden,3,320.12
3,707250,Peta Juliff,Pacocha and Sons,Paloma Sheavills,Carbonated Water - Raspberry,4,132.22
4,532894,Shaina Estick,Block LLC,Giovanni Schwier,Steampan Lid,5,520.72


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Account     1000 non-null int64
Name        1000 non-null object
Company     1000 non-null object
Manager     1000 non-null object
Product     1000 non-null object
Quantity    1000 non-null int64
Price       1000 non-null float64
dtypes: float64(1), int64(2), object(4)
memory usage: 54.8+ KB


In [6]:
sales_pivot = df.pivot_table(index=['Company','Manager','Product'],
                             values=['Quantity','Price'],
                             aggfunc=[np.sum,np.mean],
                             fill_value=0)
sales_pivot.head()

sum  \
                                                                               Price   
Company                 Manager           Product                                      
Abbott, Beer and Wisozk Damiano Lambdon   Ice Cream Bar - Oreo Cone           541.42   
Abbott, Metz and Emard  Pietra Puttan     Sprouts - Pea                       779.21   
Abbott-Borer            Anselma MacKain   Mints - Striped Red                 957.16   
Abernathy LLC           Abelard Greenrodd Prunes - Pitted                     990.19   
Abernathy and Sons      Amby Scallon      Wine - Touraine Azay - Le - Rideau  340.63   

                                                                                       \
                                                                             Quantity   
Company                 Manager           Product                                       
Abbott, Beer and Wisozk Damiano Lambdon   Ice Cream Bar - Oreo Cone               923   
Abbott, Metz and Emard  Pietra Puttan     Sprouts - Pea                           213   
Abbott-Borer            Anselma MacKain   Mints - Striped Red                     382   
Abernathy LLC           Abelard Greenrodd Prunes - Pitted                         595   
Abernathy and Sons      Amby Scallon      Wine - Touraine Azay - Le - Rideau      847   

                                                                                mean  \
                                                                               Price   
Company                 Manager           Product                                      
Abbott, Beer and Wisozk Damiano Lambdon   Ice Cream Bar - Oreo Cone           541.42   
Abbott, Metz and Emard  Pietra Puttan     Sprouts - Pea                       779.21   
Abbott-Borer            Anselma MacKain   Mints - Striped Red                 957.16   
Abernathy LLC           Abelard Greenrodd Prunes - Pitted                     990.19   
Abernathy and Sons      Amby Scallon      Wine - Touraine Azay - Le - Rideau  340.63   

                                                                                       
                                                                             Quantity  
Company                 Manager           Product                                      
Abbott, Beer and Wisozk Damiano Lambdon   Ice Cream Bar - Oreo Cone               923  
Abbott, Metz and Emard  Pietra Puttan     Sprouts - Pea                           213  
Abbott-Borer            Anselma MacKain   Mints - Striped Red                     382  
Abernathy LLC           Abelard Greenrodd Prunes - Pitted                         595  
Abernathy and Sons      Amby Scallon      Wine - Touraine Azay - Le - Rideau      847

In [7]:
df['Product'].value_counts().head()

Wine - Gato Negro Cabernet    4
Bandage - Fexible 1x3         3
Pail For Lid 1537             3
Mushroom - Crimini            3
Beef - Chuck, Boneless        3
Name: Product, dtype: int64

In [8]:
print(df[df['Product']=='Wine - Gato Negro Cabernet']['Quantity'].mean())
print(df[df['Product']=='Pork - Tenderloin, Frozen']['Price'].mean())
print(df[df['Product']=='Wine - Gato Negro Cabernet']['Quantity'].mean())
print(df[df['Product']=='Pork - Tenderloin, Frozen']['Price'].mean())

423.75
354.46666666666664
423.75
354.46666666666664


Here is a very interesting tool when you are working with pandas.
You can simples use this function, send data to clipboard and paste on excel.
```python
df.to_clipboard()
```

In [9]:
df.to_clipboard()

## Templating
Jinja is very powerfull and it's very common to use with python, it allows you to "code" inside a HTML template.

Jinja has some [builtin filters](http://jinja.pocoo.org/docs/dev/templates/#builtin-filters) that allows you to format data.

So to start let's take a look in a simple Jinja template:

```html
<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title>{{ title }}</title>
</head>
<body>
    <h2>Sales Report</h2>
     {{ sales_pivot }}
</body>
</html>
```

These variables inside **{{ }}** can be used to place your data in.

In [10]:
import jinja2
envi = jinja2.Environment(loader=jinja2.FileSystemLoader('./'))
template = envi.get_template("report_template.html")

In [11]:
template_vars = {'title' : 'Sales',
                 'sales_pivot' : sales_pivot.to_html()}

In [12]:
html_out = template.render(template_vars)

## Generate PDF

```msg
OSError: dlopen() failed to load a library: cairo / cairo-2
```

[Looks like you don't have cairo installed](https://github.com/Kozea/CairoSVG/issues/84)

In [13]:
from weasyprint import HTML

OSError: dlopen() failed to load a library: cairo / cairo-2

In [ ]:
HTML(string=html_out).write_pdf('report.pdf')